In [3]:
rm(list = ls())
setwd("C:/Users/ashar/Dropbox/PPMI")
library(PPMI)
####### Load the Prediction libraries ###################
library(xgboost)
library(randomForest)
library(glmnet)
library(mlr)
library(caret)
library(mlrHyperopt)
library(mlrMBO)
library(pROC)

##### Load the data #########
pd.complete <- read.csv(file = 'PPMI_complete.csv')
##### Load the annotation

#####Load the annotation ########
anno <- read.csv(file = 'PatientCharacteristics_PPMI.csv')
rownames(anno) <- anno$X

##### Check whether the rownames are the same
pd.complete[,1] == rownames(anno)

############# INPUT SNP data ###########################################
############# 3742 SNPs (based on some PASS filter) ###################
snp.data <- read.csv(file = 'PPMI_combPD_genotype.csv')

############# Get the names of the 
names.snp <- names(snp.data)

names.snp2 <- c(0)
names.snp2[1] <- "X"
for( i in 2:length(names.snp)){
  names.snp2[i] <- unlist(strsplit(as.character(names.snp[i]), split = 'PPMISI'))[[2]]
}

######## Check how many PD patients have corresponding SNP data available ##########
###### 354 Patients have corresponding annotation ##################################
sum((names.snp2 %in% rownames(anno))+0)

################ Rename the Variables of the SNP data ################################
names(snp.data) <- names.snp2

############### Take only those patients which have corresponding annotation ##########
snp.rel <- snp.data[names(snp.data) %in% pd.complete[,1] ]

row.names(snp.rel) <- snp.data$X

######################## Arrange the SNP data ##########
group <- anno$PD.progression.HDDC[match(colnames(snp.rel), rownames(anno))]

names(group) <- rownames(anno)[match(colnames(snp.rel), rownames(anno))]


######################## Arrange the SNP data ##########
rownames(anno) == pd.complete$X

anno.snp <- anno[match(colnames(snp.rel), rownames(anno)),]

colnames(snp.rel) == rownames(anno.snp)


########## It's better to Parse out the Risk Allele Variant from the names #############
all.snps <- c(0)
for(i in 1:dim(snp.rel)[1]){
  all.snps[i] <-  unlist(strsplit(rownames(snp.rel)[i], "_"))[1]
}

row.names(snp.rel) <- all.snps

modality1 <- t(snp.rel)

########## How can I visualize this using complex Heatmap library
# library(ComplexHeatmap)
# 
# split <- paste0("Cluster ", group)
# df <- as.data.frame(as.character(group))
# names(df) <- 'cluster'
# cols <- gray.colors(3, start = 1.0, end = 0)
# ha = HeatmapAnnotation(df = df, col = list(cluster = c("1" = "black", "2" = "red", "3" = "green")), which = "row", width = unit(2, "cm"))
# ht <- Heatmap(t(snp.rel),col = cols, split=split, cluster_columns = TRUE, show_heatmap_legend = T, show_row_names =FALSE, column_title = "SNP mutation", row_title = "SNP", name = 'SNP data', column_title_side = "top")

# pdf('SNP_viz.pdf')
# ht+ ha
# dev.off()

######### Load the SNP annotation ####################
load("C:/Users/ashar/Dropbox/PPMI/SNP/RiskSNPs.rda")

genes.pd <- names(table(combPD$GENCODE_name))

##### Load the 18 mechanism based SNPs ##########################
##### Check if ALL 18 SNPs are contained in the SNP matrix ######
subgraphPD.snps.rel <- list(NA)
for( i in 1:length(subgraphPD.snps)){
  snps <- subgraphPD.snps[[i]]
  snps.real <- snps[snps %in% rownames(snp.rel) ]
  subgraphPD.snps.rel[[i]] <- snps.real
  names(subgraphPD.snps.rel)[i] <- names(subgraphPD.snps)[i]
}

##18 Mechanisms based SNPS ##############
Y.list <- list(NA)
for( i in 1:length(subgraphPD.snps)){
  snps <- subgraphPD.snps[[i]]
  Y.list[[i]] <- t(snp.rel[subgraphPD.snps.rel[[i]],])
  names(Y.list)[i] <- names(subgraphPD.snps)[i]
}

###### Better the SNP matrix #############
##### Check if SNP columns are NA ########
Y.list.rel <- list(NA)
for( i in 1:length(Y.list)){
  indx <- c(0)
  tmp <- Y.list[[i]]
  indx <- as.numeric(which(is.na(tmp[1,])))
  if(length(indx) == dim(tmp)[2]){
    Y.list.rel[[i]] <- NA
  } else if(length(indx)==0){
    Y.list.rel[[i]] <- tmp
  } else{
    Y.list.rel[[i]] <- tmp[,-indx] 
  }
  names(Y.list.rel)[i] <- names(subgraphPD.snps)[i]
}

####### Remove the list which is NA ##################################
####### This means that there are 13 Mechanism without NA ############
Y.list.final <- list(NA)
countr = 1
for ( j in 1:length(Y.list.rel)){
  if(!is.na(Y.list.rel[[j]])){
    Y.list.final[[countr]] <- Y.list.rel[[j]]
    names(Y.list.final)[countr] <- names(Y.list.rel)[j]
    countr = countr + 1
  } 
}

modality2 <- Y.list.final


###### Check Names of List#######
subgraphPD.rel <- subgraphPD[names(Y.list.final)]


############################################################################################################
############## Let's see if the Disease SNPs are the same as downloaded from GWAS catalogue from EBI ########
#############################################################################################################
# 
# emblSNPS <- read.csv(file = "C:/Users/ashar/Dropbox/PPMI/SNP/SNPs_embl.csv")
# 
# ##### 185 disease associated SNPs ##############################
# snps.embl <- emblSNPS$SNPS[emblSNPS$SNPS %in% rownames(snp.rel)]

###### Get the disease related SNPS ####################################
riskSNPS <- read.csv(file = "C:/Users/ashar/Dropbox/PPMI/SNP/CuratedSNPs_DisGeNET_PD.csv")
snps.risk <- as.character(unique(riskSNPS$SNP[(riskSNPS$SNP %in% rownames(snp.rel))]))

########## Modality 3 #################################################
modality3 <- modality1[,colnames(modality1) %in% snps.risk]


ERROR: Error in library(xgboost): there is no package called ‘xgboost’
